### Step1. import library

In [1]:
import traitlets , time
import ipywidgets.widgets as widgets
from IPython.display import display
from jetbot import Camera, bgr8_to_jpeg
from jetbot import Robot

/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:519: FutureWarning: Passing (type, 1) or 

### Step2. define robot motion

In [2]:
robot = Robot()
speed = 0.5
sleep_time = 0.1

def stop(change):
    robot.stop()
    
def step_forward(change):
    robot.forward(speed)
    time.sleep(sleep_time)
    robot.stop()

def step_backward(change):
    robot.backward(speed)
    time.sleep(sleep_time)
    robot.stop()

def step_left(change):
    robot.left(speed)
    time.sleep(sleep_time)
    robot.stop()

def step_right(change):
    robot.right(speed)
    time.sleep(sleep_time)
    robot.stop()

### Step3. Create buttons and link motion

creat buttons to control the car.

But these buttons wont do anything whule creating. We have to attach functions to move the car. 

In [3]:
# create buttons
button_layout = widgets.Layout(width='100px', height='80px', align_self='center')
stop_button = widgets.Button(description='stop', button_style='danger', layout=button_layout)
forward_button = widgets.Button(description='forward', layout=button_layout)
backward_button = widgets.Button(description='backward', layout=button_layout)
left_button = widgets.Button(description='left', layout=button_layout)
right_button = widgets.Button(description='right', layout=button_layout)

# display buttons
middle_box = widgets.HBox([left_button, stop_button, right_button], layout=widgets.Layout(align_self='center'))
controls_box = widgets.VBox([forward_button, middle_box, backward_button])
display(controls_box)

# link buttons to actions
stop_button.on_click(stop)

forward_button.on_click(step_forward)
backward_button.on_click(step_backward)
left_button.on_click(step_left)
right_button.on_click(step_right)

### Display live camera feed

So let's get started.  First, let's initialize and display our camera like we did in the *teleoperation* notebook.  

> Our neural network takes a 224x224 pixel image as input.  We'll set our camera to that size to minimize the filesize of our dataset (we've tested that it works for this task).
> In some scenarios it may be better to collect data in a larger image size and downscale to the desired size later.

In [4]:
camera = Camera.instance(width=224, height=224)

image = widgets.Image(format='jpeg', width=224, height=224 )  # this width and height doesn't necessarily have to match the camera

camera_link = traitlets.dlink((camera, 'value'), (image, 'value'), transform=bgr8_to_jpeg)

# display(image)

Awesome, next let's create a few directories where we'll store all our data. We'll create a folder dataset that will contain two sub-folders free and blocked, where we'll place the images for each scenario.

In [5]:
import os

nonesign_dir = 'dataset/0_nonesign'
redlight_dir = 'dataset/1_redlight'
greenlight_dir = 'dataset/2_greenlight'
parkingsign_dir = 'dataset/3_parkingsign'

# we have this "try/except" statement because these next functions can throw an error if the directories exist already
try:
    os.makedirs(nonesign_dir)
    os.makedirs(redlight_dir)
    os.makedirs(greenlight_dir)
    os.makedirs(parkingsign_dir)
    
except FileExistsError:
    print('Directories not created becasue they already exist')

Directories not created becasue they already exist


If you refresh the Jupyter file browser on the left, you should now see those directories appear.  Next, let's create and display some buttons that we'll use to save snapshots
for each class label.  We'll also add some text boxes that will display how many images of each category that we've collected so far. This is useful because we want to make
sure we collect about as many ``free`` images as ``blocked`` images.  It also helps to know how many images we've collected overall.

In [1]:
!rm -rf dataset/0_nonesign/.ipynb_checkpoints

In [6]:
button_layout = widgets.Layout(width='128px', height='64px')

nonesign_button = widgets.Button(description='nonesign', layout=button_layout)
greenlight_button = widgets.Button(description='greenlight', button_style='success', layout=button_layout)
redlight_button = widgets.Button(description='redlight', button_style='danger', layout=button_layout)
parkingsign_button = widgets.Button(description='parkingsign', button_style='info', layout=button_layout)

nonesign_count = widgets.IntText(layout=button_layout, value=len(os.listdir(nonesign_dir)))
greenlight_count = widgets.IntText(layout=button_layout, value=len(os.listdir(greenlight_dir)))
redlight_count = widgets.IntText(layout=button_layout, value=len(os.listdir(redlight_dir)))
parkingsign_count = widgets.IntText(layout=button_layout, value=len(os.listdir(parkingsign_dir)))


none_H = widgets.HBox([nonesign_button, nonesign_count])
greenlight_H = widgets.HBox([greenlight_button, greenlight_count])
redlight_H = widgets.HBox([redlight_button, redlight_count])
parkingsign_H = widgets.HBox([parkingsign_button, parkingsign_count])
        
sign_botton = widgets.VBox([none_H, greenlight_H, redlight_H, parkingsign_H])
display(sign_botton)

Right now, these buttons wont do anything.  We have to attach functions to save images for each category to the buttons' ``on_click`` event.  We'll save the value
of the ``Image`` widget (rather than the camera), because it's already in compressed JPEG format!

To make sure we don't repeat any file names (even across different machines!) we'll use the ``uuid`` package in python, which defines the ``uuid1`` method to generate
a unique identifier.  This unique identifier is generated from information like the current time and the machine address.

In [7]:
from uuid import uuid1

def save_snapshot(directory):
    image_path = os.path.join(directory, str(uuid1()) + '.jpg')
    with open(image_path, 'wb') as f:
        f.write(image.value)
def save_nonesign():
    global nonesign_dir, nonesign_count
    save_snapshot(nonesign_dir)
    nonesign_count.value = len(os.listdir(nonesign_dir))
        
def save_greenlight():
    global greenlight_dir, greenlight_count
    save_snapshot(greenlight_dir)
    greenlight_count.value = len(os.listdir(greenlight_dir))
    
def save_redlight():
    global redlight_dir, redlight_count
    save_snapshot(redlight_dir)
    redlight_count.value = len(os.listdir(redlight_dir))

def save_parkingsign():
    global parkingsign_dir, parkingsign_count
    save_snapshot(parkingsign_dir)
    parkingsign_count.value = len(os.listdir(parkingsign_dir))

# attach the callbacks, we use a 'lambda' function to ignore the
# parameter that the on_click event would provide to our function
# because we don't need it.

nonesign_button.on_click(lambda x: save_nonesign())
greenlight_button.on_click(lambda x: save_greenlight())
redlight_button.on_click(lambda x: save_redlight())
parkingsign_button.on_click(lambda x: save_parkingsign())

display(widgets.HBox([image, controls_box, sign_botton]))